In [18]:
import os
import re

def leer_archivo_html(ruta_archivo):
    """
    Lee un archivo HTML y extrae solo el contenido de las etiquetas <script>
    """
    try:
        with open(ruta_archivo, 'r', encoding='utf-8') as archivo:
            contenido_completo = archivo.read()
            
        # Extraer solo el contenido entre etiquetas <script>
        import re
        script_matches = re.findall(r'<script[^>]*>(.*?)</script>', contenido_completo, re.DOTALL)
        # script_matches = re.findall(r'<style[^>]*>(.*?)</style>', contenido_completo, re.DOTALL)

        # Combinar todo el contenido de script en un solo string
        contenido_script = '\n'.join(script_matches)
        
        return contenido_script
        
    except Exception as e:
        print(f"Error leyendo archivo {ruta_archivo}: {e}")
        return ""

# VERSIÓN FINAL - EXACTAMENTE COMO PIDIÓ EL USUARIO
def detector_copias_final(carpeta_calculadoras, umbral_lineas=100):
    """
    Detecta trabajos copiados según especificaciones exactas:
    - Compara cada archivo con los demás
    - Si comparten más de 'umbral_lineas' líneas, marca como copiados
    - El umbral de líneas es configurable mediante el parámetro 'umbral_lineas' (por defecto 100)
    - Ignora espacios en blanco y mayúsculas
    - Una línea por archivo original con lista de legajos copiados
    """
    
    # Obtener todos los archivos
    archivos = []
    for archivo in os.listdir(carpeta_calculadoras):
        if archivo.endswith('.html'):
            legajo = archivo.replace('.html', '')
            ruta_completa = os.path.join(carpeta_calculadoras, archivo)
            archivos.append((legajo, ruta_completa))
    
    archivos.sort(key=lambda x: int(x[0]))
    
    # Leer y normalizar contenidos
    contenidos_normalizados = {}
    for legajo, ruta in archivos:
        contenido = leer_archivo_html(ruta)
        # Normalizar: quitar espacios y convertir a minúsculas
        lineas_normalizadas = []
        for linea in contenido.split('\n'):
            linea_norm = re.sub(r'\s+', '', linea.lower())
            if linea_norm:  # Solo líneas no vacías
                lineas_normalizadas.append(linea_norm)
        contenidos_normalizados[legajo] = lineas_normalizadas
    
    # Detectar grupos de copia
    grupos_copia = []
    procesados = set()
    
    for legajo1 in contenidos_normalizados:
        if legajo1 in procesados:
            continue
            
        grupo = [legajo1]
        lineas1 = set(contenidos_normalizados[legajo1])
        
        for legajo2 in contenidos_normalizados:
            if legajo1 != legajo2 and legajo2 not in procesados:
                lineas2 = set(contenidos_normalizados[legajo2])
                
                # Contar líneas comunes
                lineas_comunes = len(lineas1.intersection(lineas2))
                
                if lineas_comunes >= umbral_lineas:
                    grupo.append(legajo2)
        
        # Si hay copias, marcar todos como procesados
        if len(grupo) > 1:
            for legajo in grupo:
                procesados.add(legajo)
            grupos_copia.append(grupo)
        else:
            procesados.add(legajo1)
    
    return grupos_copia

def mostrar_reporte_final(grupos_copia, umbral_lineas):
    """
    Muestra el reporte final en el formato solicitado
    """
    print("=== REPORTE DE TRABAJOS COPIADOS ===")
    print(f"Umbral: {umbral_lineas}+ líneas comunes")
    print("Formato: [Original] : [Lista de copiados]\n")
    
    if not grupos_copia:
        print("✅ No se detectaron trabajos copiados")
        return
    
    # Ordenar grupos por tamaño (más copias primero)
    grupos_copia.sort(key=len, reverse=True)
    
    for i, grupo in enumerate(grupos_copia, 1):
        original = min(grupo, key=int)  # El legajo más bajo es el "original"
        copiados = [legajo for legajo in grupo if legajo != original]
        copiados.sort(key=int)
        
        copiados_str = ", ".join(copiados)
        print(f"{original}: {copiados_str}")
    
    print(f"\n=== ESTADÍSTICAS ===")
    total_grupos = len(grupos_copia)
    total_copias = sum(len(grupo) - 1 for grupo in grupos_copia)  # -1 porque excluimos el original
    total_archivos_involucrados = sum(len(grupo) for grupo in grupos_copia)
    
    print(f"Grupos de copia detectados: {total_grupos}")
    print(f"Total de copias: {total_copias}")
    print(f"Archivos involucrados en copias: {total_archivos_involucrados}")

# EJECUTAR DETECCIÓN FINAL
umbral_lineas = 50  # Reducido para detectar similitudes en JavaScript
carpeta_calculadoras = '.'  # Carpeta actual donde están los archivos HTML
print("🔍 Ejecutando detección final de trabajos copiados...")
print(f"📋 Criterio: Archivos que comparten {umbral_lineas}+ líneas idénticas")
print("⚙️  Normalización: Sin espacios, tabs, ni mayúsculas\n")

grupos = detector_copias_final(carpeta_calculadoras, umbral_lineas)
mostrar_reporte_final(grupos, umbral_lineas)

🔍 Ejecutando detección final de trabajos copiados...
📋 Criterio: Archivos que comparten 50+ líneas idénticas
⚙️  Normalización: Sin espacios, tabs, ni mayúsculas

=== REPORTE DE TRABAJOS COPIADOS ===
Umbral: 50+ líneas comunes
Formato: [Original] : [Lista de copiados]

61060: 61671, 62136
61478: 61667
61535: 61680
61627: 61794

=== ESTADÍSTICAS ===
Grupos de copia detectados: 4
Total de copias: 5
Archivos involucrados en copias: 9


In [19]:
# RESUMEN EJECUTIVO - SOLO CASOS MÁS IMPORTANTES
def resumen_ejecutivo(grupos_copia, min_grupo_size=5):
    """
    Muestra solo los casos más críticos para revisión manual
    """
    print("\n" + "="*60)
    print("🚨 RESUMEN EJECUTIVO - CASOS CRÍTICOS 🚨")
    print("="*60)
    
    if not grupos_copia:
        print("✅ No se detectaron trabajos copiados")
        return
    
    # Filtrar solo grupos grandes
    casos_criticos = [grupo for grupo in grupos_copia if len(grupo) >= min_grupo_size]
    
    if not casos_criticos:
        print(f"ℹ️  No hay grupos con {min_grupo_size}+ copias")
        print(f"📊 Grupo más grande encontrado: {max(len(g) for g in grupos_copia)} trabajos")
        return
    
    print(f"📋 Mostrando grupos con {min_grupo_size}+ trabajos involucrados:\n")
    
    for i, grupo in enumerate(casos_criticos, 1):
        original = min(grupo, key=int)
        copiados = [legajo for legajo in grupo if legajo != original]
        copiados.sort(key=int)
        
        print(f"🔥 CASO CRÍTICO #{i}")
        print(f"   📄 Original: Legajo {original}")
        print(f"   📋 Copias ({len(copiados)}): {', '.join(copiados)}")
        print()
    
    # Estadísticas generales
    print("📊 ESTADÍSTICAS GENERALES:")
    print(f"   • Total de grupos de copia: {len(grupos_copia)}")
    print(f"   • Casos críticos ({min_grupo_size}+ copias): {len(casos_criticos)}")
    print(f"   • Grupo más grande: {max(len(g) for g in grupos_copia)} trabajos")
    print(f"   • Total de estudiantes involucrados: {sum(len(g) for g in grupos_copia)}")

# Ejecutar resumen ejecutivo
resumen_ejecutivo(grupos, min_grupo_size=3)

# También mostrar algunos grupos pequeños para contexto
print("\n" + "="*40)
print("📝 MUESTRA DE GRUPOS PEQUEÑOS (2-4 copias)")
print("="*40)

grupos_pequenos = [g for g in grupos if 2 <= len(g) <= 4][:10]  # Solo primeros 10
for i, grupo in enumerate(grupos_pequenos, 1):
    original = min(grupo, key=int)
    copiados = [legajo for legajo in grupo if legajo != original]
    print(f"{i:2d}. {original}: {', '.join(sorted(copiados, key=int))}")

if len(grupos_pequenos) < len([g for g in grupos if 2 <= len(g) <= 4]):
    total_pequenos = len([g for g in grupos if 2 <= len(g) <= 4])
    print(f"    ... y {total_pequenos - 10} grupos más")


🚨 RESUMEN EJECUTIVO - CASOS CRÍTICOS 🚨
📋 Mostrando grupos con 3+ trabajos involucrados:

🔥 CASO CRÍTICO #1
   📄 Original: Legajo 61060
   📋 Copias (2): 61671, 62136

📊 ESTADÍSTICAS GENERALES:
   • Total de grupos de copia: 4
   • Casos críticos (3+ copias): 1
   • Grupo más grande: 3 trabajos
   • Total de estudiantes involucrados: 9

📝 MUESTRA DE GRUPOS PEQUEÑOS (2-4 copias)
 1. 61060: 61671, 62136
 2. 61478: 61667
 3. 61535: 61680
 4. 61627: 61794
